<a href="https://colab.research.google.com/github/endnef2/big_leader/blob/main/Task_Documentation/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [211]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ChatGPT/fra.txt", sep='\t',
            names=['src', 'tar', 'lic'])

In [212]:
df

,src,tar,lic
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
227810,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
227811,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
227812,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
227813,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [213]:
df = df[:10000]

In [214]:
# 열 제거
del df['lic']

In [215]:
df.head(5)

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


### sos, eos 표현

In [216]:
# dataframe의 target 열에 대해 해당 함수를 적용해
df.tar = df.tar.apply(lambda x: '\t' + x + '\n') # sos:\t, eos:\n

<ipython-input-216-ba5c4453baed>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.tar = df.tar.apply(lambda x: '\t' + x + '\n') # sos:\t, eos:\n


In [217]:
df.head(5)

,src,tar
0,Go.,\tVa !\n
1,Go.,\tMarche.\n
2,Go.,\tEn route !\n
3,Go.,\tBouge !\n
4,Hi.,\tSalut !\n


### 문자단위 분리

In [218]:
# 중복제거, 문자단위로 분리
srcVocab = set()
for line in df.src: # 1줄씩 읽음
    for c in line: # 1개의 문자씩 읽음
      srcVocab.add(c)

In [219]:
# encoder에서 적용될 문장들 문자 단위로 분리해서 중복값 제거
srcVocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [220]:
# decoder에서 적용될 값들
tarVocab = set()
for line in df.tar:
    for c in line:
      tarVocab.add(c)

In [221]:
tarVocab

{'\t',
 '\n',
 ' ',
 '!',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '’',
 '\u202f'}

In [222]:
len(tarVocab) # 1009

93

In [223]:
len(srcVocab) # 74

70

In [224]:
list(srcVocab)[0]
list(srcVocab)[20]

'E'

In [225]:
srcVocab = sorted(list(srcVocab))

In [226]:
tarVocab = sorted(list(tarVocab))

In [227]:
src_vocab_size = len(srcVocab)+1
tar_vocab_size = len(tarVocab)+1 # 0번 사용하지 않음

In [228]:
src_vocab_size

71

In [229]:
tar_vocab_size

94

### 숫자 인덱스 부여

In [230]:
# 1번부터 번호 부여
tarVocab
tar_to_index=dict([(w, i+1) for i, w in enumerate(tarVocab)])

In [231]:
tar_to_index

{'\t': 1,
 '\n': 2,
 ' ': 3,
 '!': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '5': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'W': 44,
 'Y': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71,
 '\xa0': 72,
 '«': 73,
 '»': 74,
 'À': 75,
 'Ç': 76,
 'É': 77,
 'Ê': 78,
 'à': 79,
 'â': 80,
 'ç': 81,
 'è': 82,
 'é': 83,
 'ê': 84,
 'î': 85,
 'ï': 86,
 'ô': 87,
 'ù': 88,
 'û': 89,
 'œ': 90,
 '\u2009': 91,
 '’': 92,
 '\u202f': 93}

In [232]:
srcVocab
src_to_index=dict([(w, i+1) for i, w in enumerate(srcVocab)])

In [233]:
src_to_index

{' ': 1,
 '!': 2,
 '"': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '5': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70}

In [234]:
src_to_index['o']

59

### 인코딩

In [235]:
enc_input=[]
for line in df.src:
    enc_line=[]
    for c in line:
        enc_line.append(src_to_index[c])
    enc_input.append(enc_line)

In [236]:
enc_input

[[27, 59, 10],
 [27, 59, 10],
 [27, 59, 10],
 [27, 59, 10],
 [28, 53, 10],
 [28, 53, 10],
 [38, 65, 58, 2],
 [38, 65, 58, 2],
 [38, 65, 58, 2],
 [38, 65, 58, 2],
 [38, 65, 58, 2],
 [38, 65, 58, 2],
 [38, 65, 58, 2],
 [38, 65, 58, 2],
 [38, 65, 58, 10],
 [38, 65, 58, 10],
 [38, 65, 58, 10],
 [38, 65, 58, 10],
 [38, 65, 58, 10],
 [38, 65, 58, 10],
 [38, 65, 58, 10],
 [38, 65, 58, 10],
 [43, 52, 59, 20],
 [43, 59, 67, 2],
 [43, 59, 67, 2],
 [43, 59, 67, 2],
 [24, 65, 47, 55, 2],
 [24, 65, 47, 55, 2],
 [24, 65, 47, 55, 2],
 [26, 53, 62, 49, 2],
 [28, 49, 56, 60, 2],
 [28, 53, 48, 49, 10],
 [28, 53, 48, 49, 10],
 [30, 65, 57, 60, 2],
 [30, 65, 57, 60, 10],
 [39, 64, 59, 60, 2],
 [39, 64, 59, 60, 2],
 [39, 64, 59, 60, 2],
 [43, 45, 53, 64, 2],
 [43, 45, 53, 64, 2],
 [43, 45, 53, 64, 2],
 [43, 45, 53, 64, 10],
 [43, 45, 53, 64, 10],
 [43, 45, 53, 64, 10],
 [43, 45, 53, 64, 10],
 [22, 49, 51, 53, 58, 10],
 [22, 49, 51, 53, 58, 10],
 [27, 59, 1, 59, 58, 10],
 [27, 59, 1, 59, 58, 10],
 [27, 59, 

In [237]:
dec_input=[]
for line in df.tar:
    dec_line=[]
    for c in line:
        dec_line.append(tar_to_index[c])
    dec_input.append(dec_line)

In [238]:
dec_input

[[1, 43, 46, 3, 4, 2],
 [1, 34, 46, 63, 48, 53, 50, 12, 2],
 [1, 26, 59, 3, 63, 60, 66, 65, 50, 3, 4, 2],
 [1, 23, 60, 66, 52, 50, 3, 4, 2],
 [1, 40, 46, 57, 66, 65, 3, 4, 2],
 [1, 40, 46, 57, 66, 65, 12, 2],
 [1, 24, 60, 66, 63, 64, 93, 4, 2],
 [1, 24, 60, 66, 63, 50, 71, 93, 4, 2],
 [1,
  37,
  63,
  50,
  59,
  50,
  71,
  3,
  67,
  60,
  64,
  3,
  55,
  46,
  58,
  47,
  50,
  64,
  3,
  79,
  3,
  67,
  60,
  64,
  3,
  48,
  60,
  66,
  64,
  3,
  4,
  2],
 [1, 27, 54, 57, 50, 3, 4, 2],
 [1, 27, 54, 57, 50, 71, 3, 4, 2],
 [1, 24, 60, 66, 63, 64, 3, 4, 2],
 [1, 27, 66, 70, 50, 71, 3, 4, 2],
 [1, 27, 66, 70, 60, 59, 64, 3, 4, 2],
 [1, 24, 60, 66, 63, 64, 93, 4, 2],
 [1, 24, 60, 66, 63, 50, 71, 93, 4, 2],
 [1,
  37,
  63,
  50,
  59,
  50,
  71,
  3,
  67,
  60,
  64,
  3,
  55,
  46,
  58,
  47,
  50,
  64,
  3,
  79,
  3,
  67,
  60,
  64,
  3,
  48,
  60,
  66,
  64,
  3,
  4,
  2],
 [1, 27, 54, 57, 50, 3, 4, 2],
 [1, 27, 54, 57, 50, 71, 3, 4, 2],
 [1, 24, 60, 66, 63, 64, 3, 4,

In [239]:
# sos:'\t'를 제거
dec_tar=[]
for line in df.tar:
    enc_line=[]
    t=0
    for c in line:
        if t>0:
            enc_line.append(tar_to_index[c])
        t=t+1
    dec_tar.append(enc_line)

In [240]:
dec_tar

[[43, 46, 3, 4, 2],
 [34, 46, 63, 48, 53, 50, 12, 2],
 [26, 59, 3, 63, 60, 66, 65, 50, 3, 4, 2],
 [23, 60, 66, 52, 50, 3, 4, 2],
 [40, 46, 57, 66, 65, 3, 4, 2],
 [40, 46, 57, 66, 65, 12, 2],
 [24, 60, 66, 63, 64, 93, 4, 2],
 [24, 60, 66, 63, 50, 71, 93, 4, 2],
 [37,
  63,
  50,
  59,
  50,
  71,
  3,
  67,
  60,
  64,
  3,
  55,
  46,
  58,
  47,
  50,
  64,
  3,
  79,
  3,
  67,
  60,
  64,
  3,
  48,
  60,
  66,
  64,
  3,
  4,
  2],
 [27, 54, 57, 50, 3, 4, 2],
 [27, 54, 57, 50, 71, 3, 4, 2],
 [24, 60, 66, 63, 64, 3, 4, 2],
 [27, 66, 70, 50, 71, 3, 4, 2],
 [27, 66, 70, 60, 59, 64, 3, 4, 2],
 [24, 60, 66, 63, 64, 93, 4, 2],
 [24, 60, 66, 63, 50, 71, 93, 4, 2],
 [37,
  63,
  50,
  59,
  50,
  71,
  3,
  67,
  60,
  64,
  3,
  55,
  46,
  58,
  47,
  50,
  64,
  3,
  79,
  3,
  67,
  60,
  64,
  3,
  48,
  60,
  66,
  64,
  3,
  4,
  2],
 [27, 54, 57, 50, 3, 4, 2],
 [27, 54, 57, 50, 71, 3, 4, 2],
 [24, 60, 66, 63, 64, 3, 4, 2],
 [27, 66, 70, 50, 71, 3, 4, 2],
 [27, 66, 70, 60, 59, 64, 3

In [241]:
# 모든 입력문장의 최대 길이
max_src_len = max([len(line) for line in df.src]) # 290(영어 문장 최대길이)
max_tar_len = max([len(line) for line in df.tar]) # 351(프랑스어 문장 최대길이)

In [242]:
max_src_len # 14
max_tar_len # 59

59

In [243]:
# 영어 데이터는 영어 샘플들끼리, 프랑스어는 프랑스어 샘플들끼리 길이를 맞추어서 패딩
encoder_input = pad_sequences(enc_input, maxlen=max_src_len, padding='post')
dec_input = pad_sequences(dec_input, maxlen=max_tar_len, padding='post')
dec_target = pad_sequences(dec_tar, maxlen=max_tar_len, padding='post')

In [244]:
df.src[1000]
df.src[0]

'Go.'

In [247]:
encoder_input[0].shape

(14,)

In [248]:
encoder_input = to_categorical(encoder_input)
dec_input = to_categorical(dec_input)
dec_target = to_categorical(dec_target)

In [251]:
src_vocab_size

71

### seq2seq

In [252]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Input
from tensorflow.keras.models import Model
import numpy as np

In [253]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [256]:
# context vector
encoder_states

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_16')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_16')>]

In [257]:
encoder_outputs

<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_16')>

In [258]:
tar_vocab_size

94

In [254]:
dec_inputs = Input(shape=(None, tar_vocab_size))
dec_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
dec_outputs, _, _= dec_lstm(dec_inputs, initial_state=encoder_states)

dec_softmax_layer = Dense(tar_vocab_size, activation='softmax')
dec_outputs = dec_softmax_layer(dec_outputs)

model = Model([encoder_inputs, dec_inputs], dec_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [259]:
dec_outputs

<KerasTensor: shape=(None, None, 94) dtype=float32 (created by layer 'dense_9')>

In [260]:
df.src[1000]

'Get going.'

In [261]:
enc_input[1000][1]

49

In [209]:
enc_input[1000][13]

array([[[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[1., 0.],
        [0., 1.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0., 1.],
        [1., 0.]],

       [[0.,

In [263]:
model.fit(x=[encoder_input, dec_input], y=dec_target, batch_size=64, epochs=1, validation_split=0.2)

125/125 [==============================] - 47s 373ms/step - loss: 1.1598 - val_loss: 1.0586


In [264]:
encoder_model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, None, 71)]        0         
                                                                 
 lstm_6 (LSTM)               [(None, 256),             335872    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 335,872
Trainable params: 335,872
Non-trainable params: 0
_________________________________________________________________


In [266]:
dec_input.shape # 10000개 문장, 문자의 길이, 문자의 종류(차원)

(10000, 59, 94)

In [269]:
encoder_input.shape # 번역 전 영어 문장

(10000, 14, 71)

In [270]:
encoder_input[3].shape

(14, 71)

In [271]:
df.src[3]

'Go.'

In [272]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [273]:
encoder_states

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_16')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm_16')>]

In [265]:
# 이전 시점의 상태들을 저장하는 텐서
dec_state_input_h = Input(shape=(256,))
dec_state_input_c = Input(shape=(256,))
dec_states_inputs = [dec_state_input_h, dec_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
dec_outputs, state_h, state_c = dec_lstm(dec_inputs, initial_state=dec_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
dec_states = [state_h, state_c]
dec_outputs = dec_softmax_layer(dec_outputs)
dec_model = Model(inputs=[dec_inputs] + dec_states_inputs, outputs=[dec_outputs] + dec_states)

In [145]:
# 입력문장 : 영문
# 정답문장 : 프랑스어
# 예측번역문장 : 프랑스어

In [274]:
# 인덱스로부터 단어를 얻을 수 있도록 만들기
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [275]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  dec_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = dec_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    dec_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(dec_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return dec_sentence

In [276]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  dec_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', df.src[seq_index])
  print('정답 문장:', df.tar[seq_index][2:len(df.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', dec_sentence[1:len(dec_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 27ms/step
-----------------------------------
입력 문장: Go.
정답 문장: ouge !
번역 문장: ee          e 
1/1 [==============================] - 0s 36ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: onjour !
번역 문장: e          e 
1/1 [==============================] - 0s 35ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: 'ai pigé !
번역 문장: e          e 
1/1 [==============================] - 0s 35ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: entre à la maison.
번역 문장: ee          e
1/1 [==============================] - 0s 23ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: n avant.
번역 문장: e           e


In [277]:
seq_index

1001